**Resources list**
1. pyspark tutorial https://github.com/pydatasg/Pydata_meetup_Nov_16/blob/master/RF_modeling_2.3_business_weimin.ipynb
2. follow this tutorial: https://www.analyticsvidhya.com/blog/2016/10/spark-dataframe-and-operations/
3. and this one: https://www.dezyre.com/apache-spark-tutorial/pyspark-tutorial
4. data engineer trick: http://nadbordrozd.github.io/blog/2016/05/22/one-weird-trick-that-will-fix-your-pyspark-schemas/
5. git http://rogerdudler.github.io/git-guide/

# Get data
## Downloand data from s3
version 1.0

In [ ]:
! cd /home/ubuntu/s3/flight_1_5

! aws s3 sync s3://flight.price/flight_1_5 . 

version 1.1

In [ ]:
! aws s3 ls s3://flight.price.11/flight_1_5 --recursive
! aws s3 sync s3://flight.price.11/flight_1_5 .

In [ ]:
! du -h

## Extract
Note: version 1.1 from 2017-05-11 onwards

In [ ]:
import zipfile
import os

# dir_in = 'C:\\s3\\20170503\\flight_1_6' # sydney to shanghai
# dir_in = 'C:\\s3\\20170503\\flight_5_1' # beijing to sydney
# dir_in = 'C:\\s3\\20170503'
dir_in = '/home/ubuntu/s3/flight_1_5'

# dir_out = 'C:\\s3\\20170503_extracted\\flight_1_6'
dir_out = '/home/ubuntu/s3/flight_1_5/extracted'
extension = ".zip"

os.chdir(dir_in) # change directory from working dir to dir with files

for subdir, dirs, files in os.walk(dir_in):
    for item in files:
        if item.endswith(extension): # check for ".zip" extension
            file_name = os.path.join(subdir, item)
#             file_name = os.path.abspath(item) # get full path of files
            zip_ref = zipfile.ZipFile(file_name) # create zipfile object
            zip_ref.extractall(dir_out) # extract file to dir
            zip_ref.close() # close file  

Move files in final_results to the main folder and delete the folder

In [ ]:
# ! cd /home/ubuntu/s3/flight_1_5/extracted
# ! mv /home/ubuntu/s3/flight_1_5/extracted/final_results/*.txt .
! find /home/ubuntu/s3/flight_1_5/extracted/final_results -name '*.txt' -exec mv {} /home/ubuntu/s3/flight_1_5/extracted \;
# -exec runs any command,  {} inserts the filename found, \; marks the end of the exec command.
! rmdir /home/ubuntu/s3/flight_1_5/extracted/final_results

http://stackoverflow.com/questions/9157138/recursively-counting-files-in-a-linux-directory
* -type f to include only files.  
* | (and not ¦) redirects find command's standard output to wc command's standard input.  
* wc (short for word count) counts newlines, words and bytes on its input (docs).  
* -l to count just newlines.
* You can also remove the -type f to include directories (and symlinks) in the count.
* It's possible this command will overcount if filenames can contain newline characters.

In [ ]:
! find /home/ubuntu/s3/flight_1_5/extracted -type f | wc -l
! du -sh

# Launch spark

In [1]:
import h2o
# import pysparkling
import os
import sys
from pyspark.sql import SparkSession
from IPython.display import display
from pyspark.sql.functions import regexp_extract, col, split, udf, trim, when, from_unixtime, unix_timestamp, minute, hour, datediff
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType, StringType, BooleanType
# from pyspark.sql.types import *
import datetime
import argparse
import json
import glob, os, shutil

from pyspark import SparkContext
sc = SparkContext()

# # # spark_home = os.environ.get('SPARK_HOME', None)
# spark_home = "C:\spark-2.1.0-bin-hadoop2.7\spark-2.1.0-bin-hadoop2.7"

# if not spark_home:

#     raise ValueError('SPARK_HOME environment variable is not set')

# sys.path.insert(0, os.path.join(spark_home, 'python'))

# sys.path.insert(0, os.path.join(spark_home, 'C:\spark-2.1.0-bin-hadoop2.7\spark-2.1.0-bin-hadoop2.7\python\lib\py4j-0.10.4-src.zip')) ## may need to adjust on your system depending on which Spark version you're using and where you installed it.

# exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

spark = SparkSession \
        .builder \
        .appName("Python Spark SQL basic example") \
        .config("spark.some.config.option", "some-value") \
        .getOrCreate()
        
spark.version

'2.1.1'

In [ ]:
# hc= H2OContext(sc).start()

In [ ]:
# h2o not working yet
# # Start H2O Context
# from pysparkling import *
# sc
# hc= H2OContext(sc).start()

## Read in json files and aggregate into a dataframe

Copy version 1.1 files to a separate folder

In [2]:
def mkfldr(fldr):
    try:
      os.makedirs(fldr)
    except:
      print("Folder already exist or some error")
    
def read_from_json_file(filename):
    with open(filename,'r') as f:
        d = json.load(f)   
    return d

In [ ]:
src_fldr = "/home/ubuntu/s3/flight_1_5/extracted"
dst_fldr = "/home/ubuntu/s3/flight_1_5/extracted11"

mkfldr(dst_fldr)

for txt_file in glob.glob(os.path.join(parent_dir, 'flight_1_5_price_2017-05-1[1-4]*.txt')):
    shutil.move(txt_file, dst_fldr)

### Append multiple json files to a single jsonl file

In [ ]:
! find -type f -name "flight_1_5_price_2017-05-10_2017-06-10*.txt"

In [ ]:
comb_fldr = '/home/ubuntu/s3/comb'
mkfldr(comb_fldr)

pq_fldr = '/home/ubuntu/s3/pq'
mkfldr(pq_fldr)

os.chdir('/home/ubuntu/s3/flight_1_5/extracted')

# json_file_name = 'flight_1_5_price_2017-05-10_2017-11-06_2_7.txt'
# pq_file_name = os.path.join(pq_fldr, json_file_name.replace('.jsonl','.parquet'))

json_file_name = 'flight_1_5_price_2017-05-10_2017-06-10*.txt'
jsonl_file_name = os.path.join(comb_fldr, json_file_name.replace('.txt','.jsonl'))

# 'flight_1_5_price_2017-05-[8|9|10]*.txt'

with open(jsonl_file_name, 'w') as outfile:
    for file in glob.glob(json_file_name):                
        d = read_from_json_file(file)                
        json.dump(d, outfile)

### Read in jsonl file, flatten it, and save as a new flat jsonl file

### Set parquet schema and create empty parquet file to append to


In [3]:
def chunkIt(seq, num):
  avg = len(seq) / float(num)
  index_start = []
  index_end = []
  last = 0.0

  while last < len(seq):
#     out.append(seq[int(last):int(last + avg)])
    index_start.append(int(last))
    index_end.append(int(last + avg))
    last += avg

  return list(zip(index_start, index_end))

# files = [os.path.join(ext_fldr, "flight_1_5_price_2017-04-06_2017-04-07_1_0.txt"), 
#         os.path.join(ext_fldr, "flight_1_5_price_2017-04-06_2017-04-07_2_7.txt")]



Set chunk size

In [4]:
ext_fldr = '/home/ubuntu/s3/flight_1_5/extracted'
file_count = len(glob.glob(os.path.join(ext_fldr, "*.txt")))
range_temp = chunkIt(range(file_count), 100)

print(file_count)
print(range_temp)
print(range_temp[0][1])

22665
[(0, 226), (226, 453), (453, 679), (679, 906), (906, 1133), (1133, 1359), (1359, 1586), (1586, 1813), (1813, 2039), (2039, 2266), (2266, 2493), (2493, 2719), (2719, 2946), (2946, 3173), (3173, 3399), (3399, 3626), (3626, 3853), (3853, 4079), (4079, 4306), (4306, 4533), (4533, 4759), (4759, 4986), (4986, 5212), (5212, 5439), (5439, 5666), (5666, 5892), (5892, 6119), (6119, 6346), (6346, 6572), (6572, 6799), (6799, 7026), (7026, 7252), (7252, 7479), (7479, 7706), (7706, 7932), (7932, 8159), (8159, 8386), (8386, 8612), (8612, 8839), (8839, 9065), (9065, 9292), (9292, 9519), (9519, 9745), (9745, 9972), (9972, 10199), (10199, 10425), (10425, 10652), (10652, 10879), (10879, 11105), (11105, 11332), (11332, 11559), (11559, 11785), (11785, 12012), (12012, 12239), (12239, 12465), (12465, 12692), (12692, 12919), (12919, 13145), (13145, 13372), (13372, 13598), (13598, 13825), (13825, 14052), (14052, 14278), (14278, 14505), (14505, 14732), (14732, 14958), (14958, 15185), (15185, 15412), (1541

Create empty parquet file

In [5]:
import json
import pandas as pd
import os
from pandas.io.json import json_normalize

pd.options.display.max_columns = 99

flat_table_list = []  # a list

ext_fldr = '/home/ubuntu/s3/flight_1_5/extracted'
comb_fldr = '/home/ubuntu/s3/comb'
# os.chdir(comb_fldr)
# file_list = os.listdir(comb_fldr)

aggDF = pd.DataFrame() #aggregate dataframe to hold all individual dataframes
count = 0

for file in glob.glob(os.path.join(ext_fldr, "*.txt"))[0:1]:
    with open(file) as f:               
        flat_table = []  # a list
        d = json.load(f)
        flight_list = json_normalize(d['flight_list'])
        basic = json_normalize(d['basic'])
        
        # create dataframe
        basic = basic.drop('search_date', 1)
        basic['tmp'] = 1
        flight_list = flight_list.drop('id', 1)
        flight_list['tmp'] = 1
        DF = pd.merge(basic, flight_list, on=['tmp'])
        DF=DF.drop('tmp', 1)
        
        # append
        aggDF = aggDF.append(DF)
        
        count = count + 1
        
        print(file+" completed. Count: " + str(count))

/home/ubuntu/s3/flight_1_5/extracted/flight_1_5_price_2017-04-23_2017-04-30_2_21.txt completed. Count: 1


In [6]:
aggDF.shape

(114, 29)

In [7]:
aggDF.columns

Index(['from_city_name', 'start_date', 'stay_days', 'table_name', 'task_id',
       'to_city_name', 'trip', 'version', 'airline_code', 'airline_codes',
       'arr_time', 'check_bag_inc', 'company', 'dep_time', 'duration',
       'flight_code', 'flight_number', 'index', 'plane', 'power', 'price',
       'price_code', 'search_date', 'span_days', 'stop', 'stop_info',
       'ticket_left', 'video', 'wifi'],
      dtype='object')

In [8]:
# force to string
# aggDF['id'] = 'Unknown'
# aggDF['video'] = False
# aggDF['wifi'] = False
# aggDF['power'] = False
aggDF

,from_city_name,start_date,stay_days,table_name,task_id,to_city_name,trip,version,airline_code,airline_codes,arr_time,check_bag_inc,company,dep_time,duration,flight_code,flight_number,index,plane,power,price,price_code,search_date,span_days,stop,stop_info,ticket_left,video,wifi
0,sydney,2017-04-30,21,flight_1_5_price,34,beijing,2,1.0,CX,"[CX, KA]",2017-04-30T22:30:00.000+08:00,False,Cathay Pacific,2017-04-30T10:05:00.000+10:00,14h25m,CX162,162,50,AIRBUS INDUSTRIE A330-300,True,1001.79,AUD,2017-04-23,0,1,Hong Kong(HKG):1h10m,0,True,False
1,sydney,2017-04-30,21,flight_1_5_price,34,beijing,2,1.0,CX,"[CX, KA]",2017-04-30T23:25:00.000+08:00,False,Cathay Pacific,2017-04-30T10:05:00.000+10:00,15h20m,CX162,162,77,AIRBUS INDUSTRIE A330-300,False,1001.79,AUD,2017-04-23,0,1,Hong Kong(HKG):2h10m,0,False,False
2,sydney,2017-04-30,21,flight_1_5_price,34,beijing,2,1.0,CX,"[CX, CX]",2017-05-22T08:45:00.000+10:00,False,Cathay Pacific,2017-05-21T15:30:00.000+08:00,15h15m,CX6115,6115,47,Airbus A321,False,952.04,AUD,2017-04-23,0,1,Hong Kong(HKG):2h30m,4,False,False
3,sydney,2017-04-30,21,flight_1_5_price,34,beijing,2,1.0,CZ,"[CZ, CZ]",2017-05-22T08:25:00.000+10:00,False,China Southern Airlines,2017-05-21T14:30:00.000+08:00,15h55m,CZ3162,3162,29,Airbus A330,False,891.44,AUD,2017-04-23,0,1,Guangzhou(CAN):3h10m,6,False,False
4,sydney,2017-04-30,21,flight_1_5_price,34,beijing,2,1.0,CX,"[CX, CX]",2017-04-30T19:20:00.000+08:00,False,Cathay Pacific,2017-04-30T07:30:00.000+10:00,13h50m,CX110,110,14,AIRBUS INDUSTRIE A330-300,True,952.04,AUD,2017-04-23,0,1,Hong Kong(HKG):0h50m,9,True,False
5,sydney,2017-04-30,21,flight_1_5_price,34,beijing,2,1.0,NZ,[NZ],2017-05-01T05:30:00.000+08:00,False,Air New Zealand,2017-04-30T19:40:00.000+10:00,11h50m,NZ3884,3884,36,AIRBUS INDUSTRIE A330-300,False,2314.16,AUD,2017-04-23,0,0,,7,False,False
6,sydney,2017-04-30,21,flight_1_5_price,34,beijing,2,1.0,VN,"[VN, VN]",2017-05-22T12:15:00.000+10:00,False,Vietnam Airlines,2017-05-21T15:45:00.000+08:00,18h30m,VN513,513,15,Boeing 787,False,1578.64,AUD,2017-04-23,0,1,Hanoi(HAN):5h30m,4,False,False
7,sydney,2017-04-30,21,flight_1_5_price,34,beijing,2,1.0,CX,"[CX, KA]",2017-05-01T11:40:00.000+08:00,False,Cathay Pacific,2017-04-30T21:55:00.000+10:00,15h45m,CX138,138,63,BOEING 777-300ER,False,1001.79,AUD,2017-04-23,0,1,Hong Kong(HKG):3h15m,0,False,False
8,sydney,2017-04-30,21,flight_1_5_price,34,beijing,2,1.0,FJ,"[FJ, SQ]",2017-05-01T07:15:00.000+08:00,False,Fiji Airways,2017-04-30T15:50:00.000+10:00,17h25m,FJ5320,5320,49,AIRBUS INDUSTRIE A330-200,False,5552.39,AUD,2017-04-23,0,1,Singapore(SIN):2h55m,0,False,False
9,sydney,2017-04-30,21,flight_1_5_price,34,beijing,2,1.0,CX,"[CX, CX]",2017-05-22T08:45:00.000+10:00,False,Cathay Pacific,2017-05-21T16:35:00.000+08:00,14h10m,CX331,331,38,AIRBUS INDUSTRIE A330-300,True,952.04,AUD,2017-04-23,0,1,Hong Kong(HKG):1h5m,9,True,False


In [9]:
sp_df_test = spark.createDataFrame(aggDF)
sp_df_test.show(2, truncate=False)

+--------------+----------+---------+----------------+-------+------------+----+-------+------------+-------------+-----------------------------+-------------+--------------+-----------------------------+--------+-----------+-------------+-----+-------------------------+-----+-------+----------+-----------+---------+----+--------------------+-----------+-----+-----+
|from_city_name|start_date|stay_days|table_name      |task_id|to_city_name|trip|version|airline_code|airline_codes|arr_time                     |check_bag_inc|company       |dep_time                     |duration|flight_code|flight_number|index|plane                    |power|price  |price_code|search_date|span_days|stop|stop_info           |ticket_left|video|wifi |
+--------------+----------+---------+----------------+-------+------------+----+-------+------------+-------------+-----------------------------+-------------+--------------+-----------------------------+--------+-----------+-------------+-----+-----------------

### Create empty dataframe and save to empty parquet file

In [22]:
# if necessary delete the parquet file
! rm -rf ~/s3/comb/flight_v1_0.pq

In [12]:
struct_v1_0 = sp_df_test.schema
emptyDF = spark.createDataFrame(sc.emptyRDD(), struct_v1_0)
# emptyDF = spark.createDataFrame(aggDF, struct_v1_0)
emptyDF.write.parquet(os.path.join(comb_fldr, "flight_v1_0.pq"))

In [13]:
len(range_temp)
for chunk_num in range(0, 2):
    print(str(range_temp[chunk_num][0]), str(range_temp[chunk_num][1]))

0 226
226 453


Stopped here 20170521.  
  
**Need to**:   
* test generating multip pandas df and read into spark and then parquet - DONE
* build loop within flight_1_5 - DONE
* build loop for different sub folders on s3

**Problem**:   
* need to figure out how to flatten nested spark dataframe - for now just do this in pandas

Check out https://www.youtube.com/watch?v=noFkYVkixPA  
and https://spark.apache.org/docs/2.1.1/sql-programming-guide.html#save-modes

In [21]:
def append_pq(df):    
    spDF = spark.createDataFrame(df, struct_v1_0)
    spDF.repartition(1).write.mode('append').parquet(os.path.join(comb_fldr, "flight_v1_0.pq"))
    print(os.path.join(comb_fldr, "flight_v1_0.pq") + " saved!")    
    
    
file_count = len(glob.glob(os.path.join(ext_fldr, "*.txt")))
range_temp = chunkIt(range(file_count), 100)

for chunk_num in range(0, 10):
    count = 0
    aggDF = pd.DataFrame() #aggregate dataframe to hold all individual dataframes

    for file in glob.glob(os.path.join(ext_fldr, "*.txt"))[range_temp[chunk_num][0]:range_temp[chunk_num][1]]:        
        with open(file) as f:                           
            d = json.load(f)
            flight_list = json_normalize(d['flight_list'])
            basic = json_normalize(d['basic'])

            # create dataframe
            basic = basic.drop('search_date', 1)
            basic['tmp'] = 1
            flight_list = flight_list.drop('id', 1)
            flight_list['tmp'] = 1
            DF = pd.merge(basic, flight_list, on=['tmp'])
            DF=DF.drop('tmp', 1)
            

            # append
            aggDF = aggDF.append(DF)
            count = count + 1
#             print(file+" completed. Count: " + str(count))        
    append_pq(aggDF)

/home/ubuntu/s3/comb/flight_v1_0.pq saved!
/home/ubuntu/s3/comb/flight_v1_0.pq saved!
/home/ubuntu/s3/comb/flight_v1_0.pq saved!
/home/ubuntu/s3/comb/flight_v1_0.pq saved!
/home/ubuntu/s3/comb/flight_v1_0.pq saved!
/home/ubuntu/s3/comb/flight_v1_0.pq saved!
/home/ubuntu/s3/comb/flight_v1_0.pq saved!
/home/ubuntu/s3/comb/flight_v1_0.pq saved!
/home/ubuntu/s3/comb/flight_v1_0.pq saved!
/home/ubuntu/s3/comb/flight_v1_0.pq saved!


In [19]:
flight = spark.read.parquet("/home/ubuntu/s3/comb/flight_v1_0.pq")

In [20]:
flight.count()

68397

In [70]:
emptyDF.write.mode('append').parquet(os.path.join(comb_fldr, "flight_v1_0.pq"))


In [ ]:
! rm -rf /home/ubuntu/s3/flight_1_5/extracted/flight_1_5_price_2017-05-10_.pq
# -f = to ignore non-existent files, never prompt
# -r = to remove directories and their contents recursively
# -v = to explain what is being done

In [ ]:
spark.createDataFrame(testDF2, struct_v1_1).show()

In [ ]:
df.show(2, truncate=False)

In [ ]:
import pyspark.sql.functions as F


df1 = spark.read.json(os.path.join(comb_fldr, "flight_1_5_price_2017-05-10_2017-06-10*.jsonl"))
df2 = spark.read.json(os.path.join(comb_fldr, "flight_1_5_price_2017-05-10_2017-11-06_2_*.jsonl"))
df = df1.unionAll(df2)




# df.printSchema()
# df.select("basic.*").show()
df.select(F.explode("flight_list")).show(200, truncate=False)
# df.select["basic"]
# flat_table = []  # a list
# d = json.load(f)
# flight_list = json_normalize(d['flight_list'])
# basic = json_normalize(d['basic'])

# # create dataframe
# basic2 = basic.drop('search_date', 1)
# basic2['tmp'] = 1
# flight_list['tmp'] = 1
# DF = pd.merge(basic2, flight_list, on=['tmp'])
# DF=DF.drop('tmp', 1)

In [ ]:
basic2 = basic.drop('search_date', 1)
basic2['tmp'] = 1
flight_list['tmp'] = 1
DF = pd.merge(basic2, flight_list, on=['tmp'])
DF=DF.drop('tmp', 1)
# DF.shape

In [ ]:
f = open("temp.jsonl", "w")

for row in DF.iterrows():
    row[1].to_json(f)
    f.write("\n")
    
f.close()

open("temp.jsonl").read()
    


In [ ]:
flight = spark.read.json("/home/ubuntu/s3/comb/temp.jsonl")
flight.printSchema()

In [ ]:
flight.count()

In [ ]:
import json

data = []
with open('/home/ubuntu/s3/comb/flight_1_5_price_2017-05-10_2017-11-06_2_7.jsonl') as f:
    for line in f:
        d = json.load(f)
        flight_list = json_normalize(d['flight_list'])
        basic = json_normalize(d['flight_list'])
        flat_table = dict()
        flat_table['basic'] = basic
        flat_table['flight'] = flight_list
        flat_table_list.append(flat_table)    
df = pd.DataFrame.from_records(data)
df.head(10)

In [ ]:
! cd /home/ubuntu/s3/comb
! head -n10 *.jsonl

In [ ]:
len(flat_table_list)

In [ ]:
df = pd.DataFrame.from_records(flat_table_list)

In [ ]:
df.head(50)

In [ ]:
print(jsonl_file_name)

Having errors. Probably due to 1 Gb RAM being too small to hold the data

In [ ]:
flight = spark.read.json(jsonl_file_name)
flight.printSchema()

In [ ]:
flight.show(100)

In [ ]:
flight.write.parquet(pq_file_name)

In [ ]:
flight.show(2)

In [ ]:
extraction_path = "/home/ubuntu/s3/flight_1_5/extracted"



d = read_from_json_file(json_file_name)

jsonl_file_name = json_file_name.replace('.json','.jsonl')

with open(jsonl_file_name,'w') as f:
    json.dump(d, f)






In [ ]:
import argparse
import json
from pyspark.sql import SparkSession

# def spark_write(spark,jsonl_file_name,parquet_file_name):
#     df = spark.read.json(jsonl_file_name)
#     df.write.parquet(parquet_file_name)
    
# def spark_print_json(spark,json_file_name):
#     d = read_from_json_file(json_file_name)
    
#     jsonl_file_name = json_file_name.replace('.json','.jsonl')
    
#     with open(jsonl_file_name,'w') as f:
#         json.dump(d, f)
    
#     df = spark.read.json(jsonl_file_name)
    
#     df.createGlobalTempView("flight")
    
#     spark.sql("SELECT fromCity,toCity,flight_leg1.departureTime, flight_leg1.departureLocation.airportLongName FROM global_temp.flight").show()
    
def read_from_json_file(filename):
    with open(filename,'r') as f:
        d = json.load(f)
        
    return d

# def pq_to_json_file(spark, pq_file_name, json_file_name):
#     df = spark.read.load(pq_file_name)
    
#     df.write.json(json_file_name)
    
# def read_data_from_pq_file(spark,filename):
#     df = spark.read.load(filename)
    
#     df.select('from_city_name','to_city_name').show()
#     #spark.sql("SELECT from_city_name,to_city_name FROM global_temp.flight").show()
    
# def main():
#     full_path_pq_file_name = "test/t.parquet"

#     spark = SparkSession \
#         .builder \
#         .appName("Python Spark SQL basic example") \
#         .config("spark.some.config.option", "some-value") \
#         .getOrCreate()
            
#     parser = argparse.ArgumentParser()
    
#     parser.add_argument('cmd',help='[json,pq,json2pq,jsonprint]')
#     parser.add_argument('--json',help='json file namejson,pq]')
#     parser.add_argument('--pq',help='parquet file name')
    
#     args = parser.parse_args()
    
#     cmd_ind = args.cmd
#     json_file_name = args.json
#     pq_file_name = args.pq
    
#     if cmd_ind == 'json2pq':
#         if json_file_name is None:
#             print("Please set the json file name by --json")
#             quit(0)
            
#         if pq_file_name is None:
#             print("Please set the pqrquet file name by --pq")
#             quit(0)
#         d = read_from_json_file(json_file_name)
        
#         jsonl_file_name = json_file_name.replace('.json','.jsonl')
        
#         with open(jsonl_file_name,'w') as f:
#             json.dump(d, f)
            
#         spark_write(spark, jsonl_file_name, pq_file_name)
#     elif cmd_ind == 'pq':
#         if pq_file_name is None:
#             print("Please set the pqrquet file name by --pq")
#             quit(0)
#         read_data_from_pq_file(spark,full_path_pq_file_name)
#     elif cmd_ind == 'pq2json':
#         if json_file_name is None:
#             print("Please set the json file name by --json")
#             quit(0)
            
#         if pq_file_name is None:
#             print("Please set the pqrquet file name by --pq")
#             quit(0)
            
#         pq_to_json_file(spark,pq_file_name,json_file_name)
        
#     elif cmd_ind== 'jsonprint':
#         if json_file_name is None:
#             print("Please set the json file name by --json")
#             quit(0)
            
#         spark_print_json(spark,json_file_name)
#     else:
#         print('invalid cmd command')
        
#     spark.stop()
    
    
# if __name__== '__main__':
#     main()
    
    
# spark-submit sp.py jsonprint --json final_flight_result_format_v1.1.json


# json_file_name = "D:\\Data Science\\pySpark\\test_schema\\final_flight_result_format_v1.1.json"
# jsonl_file_name = "C:\\s3\\20170503_jsonl\\flight_1_6.jsonl"
jsonl_file_name = "C:\\s3\\a3.json"

# d = read_from_json_file(json_file_name)

# jsonl_file_name = json_file_name.replace('.json','.jsonl')

# with open(jsonl_file_name,'w') as f:
#     json.dump(d, f)

# flight = spark.read.json(jsonl_file_name)

# df.createGlobalTempView("flight")

# spark.sql("SELECT fromCity,toCity,flight_leg1.departureTime, flight_leg1.departureLocation.airportLongName FROM global_temp.flight").show()
  
# flight.printSchema()


In [ ]:
# flight.printSchema()
# flight.drop('airline_codes').repartition(1).write.format('com.databricks.spark.csv').save('C:\\s3\\20170503_jsonl\\flight_1_6.csv', header=True)

In [ ]:
# flight.write.parquet("C:\\s3\\20170503_jsonl\\flight.parquet")

In [ ]:
# flight = spark.read.parquet("C:\\s3\\20170503_jsonl\\flight.parquet")
flight = spark.read.parquet("/home/ubuntu/parquet/flight.parquet")

In [ ]:
flight.count()

In [ ]:
flight.printSchema()
flight.show(2,truncate=False)
display(flight.count())

In [ ]:
# path_to_input = "D:\Data Science\pySpark\original_json"
# original_data = spark.read.json(sc.wholeTextFiles(path_to_input).values())

# original_data = sqlContext.read.json("D:\\Data Science\\pySpark\\original_json\\original_flight_info.txt")
# original_data.printSchema()


In [ ]:
# import json
# from pprint import pprint

# # with open("D:\\Data Science\\pySpark\\original_json\\original_flight_info.txt") as data_file:    
# #     data = json.load(data_file)

# # pprint(data)
# # # data.head()

# df = spark.read.json("D:\\Data Science\\Flight v2\\python\\final_flight_result_format_v1.1.json")
# # Displays the content of the DataFrame to stdout
# df.show()




In [ ]:
# flight = sc.textFile("C:\\s3_temp\\summary\\flat_table.csv").map(lambda line: (line.split(',')[5], line.split(',')[10])).collect()
# flight = sc.textFile("C:\\s3_temp\\summary\\flat_table.csv")
# flight = spark.read.csv("C:\\s3_temp\\summary\\flat_table.csv", header=True, mode="DROPMALFORMED")

# flight.cache()

In [ ]:
# flight.repartition(1).write.format('com.databricks.spark.csv').save('D://Data Science//pySpark//test2.csv', header=True)


In [ ]:
flight.first()

In [ ]:
# flight.rdd.takeSample(False, 3, 1234)

In [ ]:
flight.dtypes

In [ ]:
# from pyspark.sql.functions import year, month, dayofmonth, hour, minute, weekofyear, crosstab
# flight2.select(year("arr_time").alias('year'), 
# from pyspark.sql.functions import regexp_extract, col, split
# # import re

# from pyspark.sql.functions import udf
# from pyspark.sql.types import IntegerType, StringType, BooleanType

def toBool(aString):
    if aString.lower() == 'true':
        return True
    else: 
        return False
 
udfToBool = udf(toBool, BooleanType())


def groupTicketLeft(aString):
    if aString == '0':
        return 'plenty'
    else: 
        return aString
 
udfGroupTicketLeft = udf(groupTicketLeft, StringType())

# def getMinutes(aString):
#     return minute(unix_timestamp(aString, "HH'h'mm'm'").cast("timestamp"))

# udfGetMinutes = udf(getMinutes, IntegerType())

# def getHours(x):
#   return re.match('([0-9]+(?=h))', x)
# getHours("14h")
# len(flight.columns)
# flight.columns
# flight.dtypes
flight2 = (flight.withColumn('search_date', flight.search_date.cast('timestamp'))
#           .withColumn('search_date_y', flight.search_date_y.cast('timestamp'))
#           .withColumnRenamed('search_date_y', 'search_date')
          .withColumn('stay_days', flight.stay_days.cast('int'))                                
           #create local time first
           .withColumn('dep_time_local', flight.dep_time.substr(1, 23).cast('timestamp'))                      
          .withColumn('dep_time', flight.dep_time.cast('timestamp'))
           #create local time first
          .withColumn('arr_time_local', flight.arr_time.substr(1, 23).cast('timestamp'))                      
          .withColumn('arr_time', flight.arr_time.cast('timestamp'))
           #duration           
           .withColumn('duration_h',split(flight.duration,'h').getItem(0))
           .withColumn('duration_m',split(flight.duration,'h').getItem(1))
           #stop info
           .withColumn('stop_info1',split(flight.stop_info,';').getItem(0))
           .withColumn('stop_info2',split(flight.stop_info,';').getItem(1))
#            .withColumn('druation_hours', 
#                        flight.selectExpr("duration", "regexp_extract(duration,'([0-9]+(?=h))', 1) as duration_hours")
#                        .duration_hours
#                        .cast('int'))
          .withColumn('start_date', flight.start_date.cast('date'))           
          .withColumn('price', flight.price.cast('double'))          
           # to correct the name
          .withColumnRenamed('check_bag_inc', 'check_bag_not_inc')
#           .withColumn('check_bag_not_inc', udfToBool(flight.check_bag_inc))
          .withColumn('power', flight.power.cast('boolean'))           
          .withColumn('video', flight.video.cast('boolean'))
          .withColumn('wifi', flight.wifi.cast('boolean'))           
          .withColumn('stop', flight.stop.cast('int')) 
          .withColumn('span_days', flight.span_days.cast('int'))           
           .withColumn('ticket_left', udfGroupTicketLeft(flight.ticket_left))
           .drop('search_date_x', 'check_bag_inc')
          )

flight2 = (flight2.withColumn('stop_loc1',split(flight2.stop_info1,':').getItem(0))
           .withColumn('stop_duration1',split(flight2.stop_info1, ':').getItem(1))
           .withColumn('stop_loc2',split(flight2.stop_info2,':').getItem(0))
           .withColumn('stop_duration2',split(flight2.stop_info2, ':').getItem(1)))

flight2 = (flight2.withColumn('stop_duration_h1',split(flight2.stop_duration1,'h').getItem(0))
           .withColumn('stop_duration_m1',split(flight2.stop_duration1,'h').getItem(1))
           .withColumn('stop_duration_h2',split(flight2.stop_duration2,'h').getItem(0))
           .withColumn('stop_duration_m2',split(flight2.stop_duration2,'h').getItem(1)))
# .withColumn('arr_time_zone', flight.arr_time.substr(24, 6))

In [ ]:
# Define udf

def getMinutes(hString, minString):
    if (hString is not None) & (minString is not None): return int(hString) * 60 + int(minString[:-1])
    else: return None
#     if score >= 80: return 'A'
#     elif score >= 60: return 'B'
#     elif score >= 35: return 'C'
#     else: return 'D'
 
udfGetMinutes = udf(getMinutes, IntegerType())
flight2 = (flight2.withColumn("duration_minutes", udfGetMinutes("duration_h", "duration_m"))
                    .withColumn("stop1_minutes", udfGetMinutes("stop_duration_h1", "stop_duration_m1"))
                   .withColumn("stop2_minutes", udfGetMinutes("stop_duration_h2", "stop_duration_m2")))
                 
flight2.show(2)

In [ ]:
# flight2.groupby('search_date', 'from_city_name', 'to_city_name', 'trip').count().show()
# flight2.groupby('table_name').count().show()
flight2.groupby('stop_info', 
                'stop_info1', 'stop_loc1', 
                'stop_duration1', 'stop_duration_h1', 'stop_duration_m1',
                'stop_info2', 'stop_loc2', 
                'stop_duration2',  'stop_duration_h2', 'stop_duration_m2').count().show(500, truncate=False)
# flight2.groupby('stop_duration1').count().show()

In [ ]:
flight2.dtypes

In [ ]:
# flight2 = flight2.drop('stop_duration_h1', 'stop_duration_m1', 'stop_duration_h2', 'stop_duration_m2',
#                       'stop_duration1', 'stop_duration2',
#                       'duration_h', 'duration_m', 
#                        'stop_info', 'stop_info1', 'stop_info2')
# display(flight2.show(2))
# display(flight2.dtypes)

In [ ]:
flight2.show(2)

In [ ]:
def groupTime(dt):
    h = dt.hour
    if h < 5 : return 'early morning'    
    elif h < 12 : return 'morning'
    elif h < 18 : return 'afternoon'    
    else: return 'evening'
#     if h >= 18 or h < 7 : return 'night'    
#     elif 7 <= h < 12 : return 'morning'
#     elif 12 <= h < 18 : return 'afternoon'    
#     else: return 'error'
    
    
udfGroupTime = udf(groupTime, StringType())
flight2 = flight2.withColumn("arr_time_group", udfGroupTime("arr_time_local")) \
                .withColumn("dep_time_group", udfGroupTime("dep_time_local"))              
flight2.cache()

In [ ]:
import pyspark.sql.functions as F

# flight2.select("dep_time_local", "dep_time_group", "arr_time_local", "arr_time_group").show(10)
#flight2.describe(['price']).show()# groupBy("dep_time_group", "arr_time_group")
# flight2.freqItems(flight2.trip)
# flight2.groupby(flight2.trip, flight2.stay_days, flight.dep_time_group).agg(func.mean('price')).show()
# flight2.groupby(flight2.trip, flight2.dep_time_group).agg(func.mean('price')).orderBy(['trip', 'avg(price)'], ascending=[1, 0]).show()
# flight2.groupby(flight2.trip, flight2.company).agg(F.mean('price'), F.stddev('price'), F.max('price'), F.min('price')).orderBy(['trip', 'stddev_samp(price)'], ascending=[1, 0]).show(10000)
flight2.filter(flight2.price <= 0).count() / flight2.count()

# drop rows with zero price
flight2 = flight2.filter(flight2.price > 0)

In [ ]:
# duration_test.select(trim(col("duration"))).show()
# df
# .withColumn('Created-formatted',when((df.Created.isNull() | (df.Created == '')) ,'0')
# .otherwise(unix_timestamp(df.Created,'yyyy-MM-dd')))

duration_test = flight2.select("stop_duration1")
duration_test.show()

duration_test.withColumn('duration_h', when(duration_test.stop_duration1.isNull(), None)
                          .otherwise(hour(unix_timestamp(duration_test.stop_duration1,"HH'h'mm'm'").cast("timestamp")))).show(20)          
    
#     .withColumn('duration_m', minute(unix_timestamp(duration_test.duration,"HH'h'mm'm'").cast("timestamp")))).show(2)


# duration_test.withColumn('duration2',when((duration_test.duration.isNull() | (duration_test.duration == '')) , 0)\
#   .otherwise(unix_timestamp(duration_test.duration,"HH'h'mm'm'").cast("timestamp"))).show(2)

In [ ]:
# from datetime import date

# import holidays

# us_holidays = holidays.UnitedStates()  # or holidays.US()

# date(2015, 1, 1) in us_holidays  # True
# date(2020, 12, 25) in us_holidays  # False

# # print(us_holidays)


# for date, name in sorted(holidays.AU(state='NSW', years=2017).items()):
#      print(date, name)
        
# for date, name in sorted(holidays.NZ(state='AUK', years=2017).items()):
#      print(date, name)

In [ ]:


# # The Holiday class will also recognize strings of any format
# # and int/float representing a Unix timestamp
# '2014-01-01' in us_holidays  # True
# '1/1/2014' in us_holidays    # True
# 1388597445 in us_holidays    # True

# us_holidays.get('2014-01-01')  # "New Year's Day"

# # Easily create custom Holiday objects with your own dates instead
# # of using the pre-defined countries/states/provinces available
# custom_holidays = holidays.HolidayBase()
# # Append custom holiday dates by passing:
# # 1) a dict with date/name key/value pairs,
# custom_holidays.append({"2015-01-01": "New Year's Day"})
# # 2) a list of dates (in any format: date, datetime, string, integer),
# custom_holidays.append(['2015-07-01', '07/04/2015'])
# # 3) a single date item
# custom_holidays.append(date(2015, 12, 25))

# date(2015, 1, 1) in custom_holidays  # True
# date(2015, 1, 2) in custom_holidays  # False
# '12/25/2015' in custom_holidays      # True

# # For more complex logic like 4th Monday of January, you can inherit the
# # HolidayBase class and define your own _populate(year) method. See below
# # documentation for examples.

In [ ]:
# from datetime import date
# from workalendar.europe import France
# cal = France()
# cal.holidays(2017)


In [ ]:
# flight2.show(2)
flight3 = flight2


def getWeekday(dt):
    return dt.weekday()
    
def getWeeknumber(dt):
    return dt.isocalendar()[1]

    
udfGetWeekday = udf(getWeekday, StringType())
udfGetWeeknumber = udf(getWeeknumber, IntegerType())

flight3 = flight3.withColumn("dep_weekday", udfGetWeekday("start_date")) \
                .withColumn("dep_weeknum", udfGetWeeknumber("start_date")) \
                .withColumn("lead_time", datediff(flight3.start_date, flight3.search_date)) 
#                 .drop('start_date', 'search_date')

flight3.show(2)
display(flight2.count(), flight3.count())
flight3.describe('lead_time').show()

In [ ]:
flight3.show(5)
flight3.dtypes

get distance to nearest holiday: http://stackoverflow.com/questions/40752378/spark-sql-distance-to-nearest-holiday
    

In [ ]:


# datetime.datetime('2017-09-01')
# date1 = datetime.datetime.strptime("2015-01-30", "%Y-%m-%d").strftime("%d-%m-%Y")
# print(date1)
# start_date_test = flight3.select("start_date").distinct()
flight3 = flight3.withColumn("start_date_str", flight3.start_date.cast('string'))

holidays = ['2017-01-01', '2017-01-02', '2017-01-27', '2017-01-28',
            '2017-01-29', '2017-01-30', '2017-01-31', '2017-02-01', '2017-02-02',
            '2017-04-02', '2017-04-03', '2017-04-04',
            '2017-05-01', '2017-05-28', '2017-05-29', '2017-05-30',
            '2017-10-01', '2017-10-02', '2017-10-03', '2017-10-04', '217-10-05', '2017-10-06']
 

index = spark.sparkContext.broadcast(sorted(holidays))

def last_holiday(date):
    last_holiday = index.value[0]
    for next_holiday in index.value:
        if next_holiday >= date:
            break
        last_holiday = next_holiday
    if last_holiday > date:
        last_holiday = None
    if next_holiday < date:
        next_holiday = None        
    return last_holiday

def next_holiday(date):
    last_holiday = index.value[0]
    for next_holiday in index.value:
        if next_holiday >= date:
            break
        last_holiday = next_holiday
    if last_holiday > date:
        last_holiday = None
    if next_holiday < date:
        next_holiday = None        
    return next_holiday


# return_type = StructType([StructField('last_holiday', StringType()), StructField('next_holiday', StringType())])

last_holiday_udf = udf(last_holiday, StringType())
next_holiday_udf = udf(next_holiday, StringType())

flight4 = flight3.withColumn('last_holiday', last_holiday_udf('start_date_str'))
flight4 = flight4.withColumn('next_holiday', next_holiday_udf('start_date_str'))
flight4 = flight4.withColumn('days_to_last_holiday', datediff('last_holiday', 'start_date_str'))
flight4 = flight4.withColumn('days_to_next_holiday', datediff('next_holiday', 'start_date_str'))

In [ ]:
(flight4
    .select('days_to_last_holiday', 'days_to_next_holiday', 'last_holiday', 'next_holiday', 'start_date')    
    .distinct().show())

In [ ]:
# flight4.show()
#'table_name', 'trip', 'stay_days'
# flight5 = (flight4.filter((flight4.trip==1) & (flight4.start_date=='2017-10-01') & (flight4.company=='AirAsiaX'))
#          .select('start_date', 'company', 'dep_time_local', 'stop_info', 'duration', 'search_date', 'price')
#          .sort(F.desc('start_date'), 'company', 'dep_time_local', 'stop_info', 'duration', 'search_date'))       
         
flight5 = flight4.sort('table_name', 'trip', 'stay_days', 
                       'start_date', 'company', 'dep_time_local',
                       'stop_info', 'duration', 'search_date')       
    
byVar = ['table_name', 'trip', 'stay_days', 'start_date', 'company', 'dep_time_local', 'stop_info', 'duration']


from pyspark.sql.window import Window

Threshold = 5

w =  Window.partitionBy(byVar).orderBy('search_date').rowsBetween(0, sys.maxsize)
flight6 = (flight5.withColumn('future_min_price', F.min(col('price')).over(w))
          .withColumn('price_will_drop',                       
                      (col('price') - col('future_min_price')) > Threshold ))
flight6 = flight6.withColumn('price_will_drop_num', flight6.price_will_drop.cast('int'))
flight6.cache()
# min_price = (flight5
#                   .groupBy(byVar)
#                   .agg(F.min(col("price"))).alias("min_price"))

# flight6 = flight5.join(min_price, on=byVar, how='left')
# # flight6.select('price', 'min(price)').show(10)
# # flight6.select('price', 'min(price)').show()
# flight6 = flight6.withColumn('price_diff', col('price')-col('min(price)'))
# flight6.show()

In [ ]:
flight6.filter((flight6.trip==1) & \
               (flight6.start_date=='2017-10-01') & \
               (flight6.company=='AirAsiaX')) \
    .select('duration', 'search_date', 'price', 'future_min_price', 'price_will_drop') \
    .show(100)

In [ ]:
flight2.unpersist()

In [ ]:
flight6.cache()

In [ ]:
# ['table_name', 'trip', 'stay_days', 'start_date', 'company', 'dep_time_local', 'stop_info', 'duration']
# groupby(flight2.trip, flight2.stay_days, flight.dep_time_group).agg(func.mean('price')).show()
# flight6.groupBy(col('trip'), col('stay_days'), col('lead_time')).agg(F.mean('price_will_drop_num')).show()
flight6.groupBy(col('trip'), col('stay_days'), col('lead_time')). \
    agg(F.mean('price_will_drop_num')).orderBy(['trip', 'stay_days', 'lead_time'], ascending=[1, 1, 0]).show(1000)


# Save to parquet

In [ ]:
flight6.coalesce(2).write.parquet("C:\\s3\\20170503_jsonl\\flight6.parquet")

In [ ]:
# flight6 = spark.read.parquet("C:\\s3\\20170503_jsonl\\flight6.parquet")
flight6 = spark.read.parquet("/home/ubuntu/parquet/flight6.parquet")


In [ ]:
flight6.count()

In [ ]:
flight6.printSchema()

In [ ]:
flight7 = (flight6.withColumn('price_drop_amt', col('price') - col('future_min_price'))
                  .withColumn('stop_minutes', col('stop1_minutes') + col('stop2_minutes'))
                  .withColumn('few_tickets_left', col('ticket_left') != '0')
                  .select('price_will_drop_num', 'price', 'few_tickets_left', 'from_city_name', 'to_city_name', 'trip', 'stay_days',
                          'company', 'flight_code', 'plane', 'span_days', 'stop',
                         'power', 'video', 'wifi', 'check_bag_not_inc', 
                         'dep_time_group', 'arr_time_group',  
                         'lead_time', 'dep_weekday', 'dep_weeknum', 
                         'days_to_last_holiday', 'days_to_next_holiday',                         
                         'duration_minutes', 'stop_minutes'
                      ).filter(flight6.price > 0))
flight7.dtypes
flight7 = flight7.na.fill({'stop_minutes': 0, 'days_to_next_holiday': 999})
flight7.show(2, truncate=False)

How to transform categorical variables? See
http://stackoverflow.com/questions/32982425/encode-and-assemble-multiple-features-in-pyspark


# One hot encoding

In [ ]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoder

column_vec_in = ['from_city_name', 'to_city_name', 'trip', 'company', 'flight_code', 'plane', 
                 'arr_time_group', 'dep_time_group', 'dep_weekday']
column_vec_out = ['from_city_name_catVec','to_city_name_catVec', 'trip_catVec',
                'company_catVec', 'flight_code_catVec', 'plane_catVec', 
                  'arr_time_group_catVec', 'dep_time_group_catVec', 'dep_weekday_catVec']
 
indexers = [StringIndexer(inputCol=x, outputCol=x+'_tmp') for x in column_vec_in ]
 
encoders = [OneHotEncoder(dropLast=False, inputCol=x+"_tmp", outputCol=y)
            for x,y in zip(column_vec_in, column_vec_out)]

tmp = [[i,j] for i,j in zip(indexers, encoders)]
tmp = [i for sublist in tmp for i in sublist]


# Prepare features and labels

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorIndexer, VectorAssembler, SQLTransformer

# prepare labeled sets
# 'price_will_drop',

cols_now = ['price', 'few_tickets_left', 'stay_days',
              'span_days', 'stop',
             'power', 'video', 'wifi', 'check_bag_not_inc',                          
             'lead_time', 'dep_weeknum', 
             'days_to_last_holiday', 'days_to_next_holiday',                         
             'duration_minutes', 'stop_minutes',
            'from_city_name_catVec','to_city_name_catVec', 'trip_catVec',
            'company_catVec', 'flight_code_catVec', 'plane_catVec', 
            'arr_time_group_catVec', 'dep_time_group_catVec', 'dep_weekday_catVec']

assembler_features = VectorAssembler(inputCols=cols_now, outputCol='features')
# labelIndexer = StringIndexer(inputCol='price', outputCol="label")
# tmp += [assembler_features, labelIndexer]
tmp += [assembler_features]
pipeline = Pipeline(stages=tmp)

In [ ]:
# flight3.na.drop().count()

# from pyspark.ml import Pipeline
# from pyspark.ml.feature import StringIndexer

# indexers = [StringIndexer(inputCol=column, outputCol=column+"_index")\
#             .fit(flight3) for column in list(set(flight3.columns)-\
#                                              set(['stay_days', 'power', 'price', 'span_days',\
#                                                   'stop', 'video', 'wifi', 'duration_minutes',\
#                                                  'dep_weeknum', 'lead_time']))]

# pipeline = Pipeline(stages=indexers)
# flight4 = pipeline.fit(flight3).transform(flight3)

# flight4.show()

In [ ]:
cols = ['from_city_name', 'to_city_name', 'trip', 'company', 'flight_code', 'plane', 
                 'arr_time_group', 'dep_time_group', 'dep_weekday']

for c in cols:
    flight7.groupBy(col(c)).count().show(50000)
    
    
flight7 = flight7.na.replace('', 'unknown', 'plane')

# http://stackoverflow.com/questions/40711229/pyspark-ml-error-urequirement-failed-cannot-have-an-empty-string-for-name
# replacements = {
#   'plane': 'unknown', 'another_col': 'another_replacement',
#   'numeric_column_wont_be_replaced': 1.0
# }

# for k, v in replacements.items():
#     # We can replace string only if target is string
#     # In Python 2 str -> basestring
#     if isinstance(v, str):
#         df = df.na.replace("", v, [k])        

In [ ]:
cols = ['price_will_drop', 'price', 'few_tickets_left', 'from_city_name', 'to_city_name', 'trip', 'stay_days',
                          'company', 'flight_code', 'plane', 'span_days', 'stop',
                         'power', 'video', 'wifi', 'check_bag_not_inc', 
                         'dep_time_group', 'arr_time_group',  
                         'lead_time', 'dep_weekday', 'dep_weeknum', 
                         'days_to_last_holiday', 'days_to_next_holiday',                         
                         'duration_minutes', 'stop_minutes']

for c in cols:
    display(c + " :" + str(flight7.where(col(c).isNull()).count()))

In [ ]:
flight7.show(2)
flight7.count()

In [ ]:
# flight7 = flight7.na.drop()
output = pipeline.fit(flight7).transform(flight7)
output = output.withColumnRenamed('price_will_drop_num', 'label')
output.cache()

In [ ]:
output.show(2)

In [ ]:
# flight4.columns

In [ ]:
# flight4.select('stay_days',
#  'power', 
#  'span_days',
#  'stop',
#  'video',
#  'wifi',
#  'duration_minutes',
#  'dep_weeknum',
#  'lead_time',
#  'trip_index',
#  'arr_time_group_index',
#  'to_city_name_index',
#  'ticket_left_index',
#  'company_index',
#  'from_city_name_index',
#  'airline_code_index',
#  'plane_index',
#  'check_bag_not_inc_index',
#  'dep_weekday_index',
#  'dep_time_group_index').dtypes

In [ ]:
# from pyspark.ml.linalg import Vectors
# from pyspark.ml.feature import VectorAssembler

# # from pyspark.ml.feature import VectorAssembler

# # ignore = ['id', 'label', 'binomial_label']
# # assembler = VectorAssembler(
# #     inputCols=[x for x in df.columns if x not in ignore],
# #     outputCol='features')

# # assembler.transform(df)
# # arr_time_group

# assembler = VectorAssembler(    
#     inputCols=[
#  'stay_days',
#  'power', 
#  'span_days',
#  'stop',
#  'video',
#  'wifi',
#  'duration_minutes', 
#  'dep_weeknum',
#  'lead_time',
#  'trip_index',
#  'arr_time_group_index',
#  'to_city_name_index',
#  'ticket_left_index',
#  'company_index',
#  'from_city_name_index',
#  'airline_code_index',
#  'plane_index',
#  'check_bag_not_inc_index',
#  'dep_weekday_index',
#  'dep_time_group_index'],
#     outputCol="features")

# output = assembler.transform(flight4)
# # print("Assembled columns 'hour', 'mobile', 'userFeatures' to vector column 'features'")
# # output.select("features", "clicked").show(truncate=False)
# output = output.withColumnRenamed('price', 'label')
# output.show(5, truncate=False)

In [ ]:
display(output.count(), flight6.count(), flight7.count())

In [ ]:
# output.coalesce(2).write.parquet("C:\\s3\\20170503_jsonl\\output.parquet")
output.coalesce(2).write.parquet("/home/ubuntu/parquet/output.parquet/output.parquet")

In [ ]:
output = spark.read.parquet("/home/ubuntu/parquet/output.parquet/output.parquet")

In [ ]:
output.count()

In [ ]:
output = output.sample(False, 0.1, seed=0)

In [ ]:
output.count()
output.printSchema()
output.show(2)

# interlude to model in h2o (not sparkling water)  
When converting the whole dataset to pandas dataframe, there was not enough RAM, JAVA was forced to shutdown!!!

In [ ]:
output.printSchema()
output2 = output.select('label', 'features')
# output2 = output.select('label', 'features').rdd

In [ ]:
output2.count()

In [ ]:
# output2.count()
flight_pd = output.select('label', 'price', 'few_tickets_left', 
                          'from_city_name', 'to_city_name', 'trip', 'stay_days',
                          'company', 'flight_code', 'plane', 'span_days', 'stop',
                         'power', 'video', 'wifi', 'check_bag_not_inc', 
                         'dep_time_group', 'arr_time_group',  
                         'lead_time', 'dep_weekday', 'dep_weeknum', 
                         'days_to_last_holiday', 'days_to_next_holiday',                         
                         'duration_minutes', 'stop_minutes').toPandas()
flight_pd.head()

In [ ]:
# flight_pd.to_csv('C:\\s3\\20170503_jsonl\\flight_pd.csv', sep='\t')
flight_pd.to_csv('/home/ubuntu/parquet/flight_pd.csv', sep='\t')

In [ ]:
h2o.init()
# from h2o.estimators.gbm import H2OGradientBoostingEstimator

# hf = h2o.H2OFrame(flight_pd)

In [ ]:
import pandas as pd
flight_pd=pd.read_csv("/home/ubuntu/parquet/flight_pd.csv", delimiter='\t')

In [ ]:
# flight_pd.head()
# help(pd.read_csv)
flight_hex = h2o.H2OFrame(flight_pd)

In [ ]:
train, valid, test = flight_hex.split_frame([0.6, 0.2], seed=1234)

flight_X = flight_hex.col_names[2:]
flight_y = flight_hex.col_names[1]    


In [ ]:
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators.random_forest import H2ORandomForestEstimator

rf_v1 = H2ORandomForestEstimator(
    model_id="rf_v1",
    ntrees=200,
    stopping_rounds=2,
    score_each_iteration=True,
    seed=1000000)

rf_v1.train(flight_X, flight_y, training_frame=train, validation_frame=valid)
rf_v1
rf_v1.score_history()
model_path = h2o.save_model(model=rf_v1, path="C:\\s3\\20170503_jsonl", force=True)
print(model_path)

In [ ]:
rf_v1

In [ ]:
# output2.take(2)
(trainingData, testData) = output2.randomSplit([0.7, 0.3])

# Stopped here! 20170513

In [ ]:
from time import time
from pyspark.ml.classification import LogisticRegression

# Load training data
# training = spark.read.format("libsvm").load("data/mllib/sample_libsvm_data.txt")

lr = LogisticRegression(maxIter=100, regParam=0.1, elasticNetParam=0.8)

# Fit the model
t0 = time()
lrModel = lr.fit(trainingData)
tt = time() - t0

# Print the coefficients and intercept for logistic regression
print("Classifier trained in " + str(tt) + " seconds.")
print("Coefficients: " + str(lrModel.coefficients))
print("Intercept: " + str(lrModel.intercept))

In [ ]:
from pyspark.ml.classification import LogisticRegression

# Extract the summary from the returned LogisticRegressionModel instance trained
# in the earlier example
trainingSummary = lrModel.summary

# Obtain the objective per iteration
objectiveHistory = trainingSummary.objectiveHistory
print("objectiveHistory:")
for objective in objectiveHistory:
    print(objective)

# Obtain the receiver-operating characteristic as a dataframe and areaUnderROC.
trainingSummary.roc.show()
print("areaUnderROC: " + str(trainingSummary.areaUnderROC))

# Set the model threshold to maximize F-Measure
fMeasure = trainingSummary.fMeasureByThreshold
maxFMeasure = fMeasure.groupBy().max('F-Measure').select('max(F-Measure)').head()
bestThreshold = fMeasure.where(fMeasure['F-Measure'] == maxFMeasure['max(F-Measure)']) \
    .select('threshold').head()['threshold']
lr.setThreshold(bestThreshold)

In [ ]:
# Make predictions.
predictions = lrModel.transform(testData)

# Select example rows to display.
predictions.select("prediction", "label", "features").show(5)
predictions.select('label', 'prediction').coalesce(1).write.csv('D://Data Science//pySpark//check_pred7.csv')

In [ ]:
from pyspark.ml import Pipeline
# from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.feature import VectorIndexer
# from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Load and parse the data file, converting it to a DataFrame.
# data = spark.read.format("libsvm").load("data/mllib/sample_libsvm_data.txt")

# Automatically identify categorical features, and index them.
# Set maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=100).fit(output)
    
# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = output.randomSplit([0.7, 0.3])

# Train a RandomForest model.
# rf = RandomForestRegressor(featuresCol="indexedFeatures", numTrees=1000, featureSubsetStrategy="auto",
#                                     impurity='variance', maxDepth=4, maxBins=32)
gbt = GBTClassifier(labelCol="label", featuresCol="indexedFeatures", maxIter=10)

# Chain indexer and forest in a Pipeline
pipeline = Pipeline(stages=[featureIndexer, gbt])

# Train model.  This also runs the indexer.
model = pipeline.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)

# Select example rows to display.
predictions.select("prediction", "label", "features").show(5)

# Select (prediction, true label) and compute test error
# evaluator = RegressionEvaluator(
#     labelCol="label", predictionCol="prediction", metricName="rmse")
evaluator = BinaryClassificationEvaluator(
    labelCol="label", predictionCol="prediction")
auc = evaluator.evaluate(predictions)
print("Area under the curve (AUC) on test data = %g" % auc)

gbtModel = model.stages[1]
print(gbtModel)  # summary only

In [ ]:
from pyspark.ml.classification import LogisticRegression

# Load training data
# training = spark.read.format("libsvm").load("data/mllib/sample_libsvm_data.txt")
flight7.withColumnRenamed('price_will_drop_num', 'label')

lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)

# Fit the model
lrModel = lr.fit(training)

# Print the coefficients and intercept for logistic regression
print("Coefficients: " + str(lrModel.coefficients))
print("Intercept: " + str(lrModel.intercept))

how to score model manually: http://stackoverflow.com/questions/35731140/apply-model-scores-to-spark-dataframe-python

In [ ]:
predictions.select('label', 'prediction').coalesce(1).write.csv('D://Data Science//pySpark//check_pred6.csv')

In [ ]:
predictions.show(20)

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

# Load and parse the data file, converting it to a DataFrame.
data = flight3

# Automatically identify categorical features, and index them.
# Set maxCategories so features with > 4 distinct values are treated as continuous.
# featureIndexer =\
#     VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(data)


# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.7, 0.3])

# Train a GBT model.
gbt = GBTRegressor(featuresCol="indexedFeatures", maxIter=10)

# Chain indexer and GBT in a Pipeline
pipeline = Pipeline(stages=[featureIndexer, gbt])

# Train model.  This also runs the indexer.
model = pipeline.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)

# Select example rows to display.
predictions.select("prediction", "label", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

gbtModel = model.stages[1]
print(gbtModel)  # summary only



In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

# # Load and parse the data file, converting it to a DataFrame.
# data = spark.read.format("libsvm").load("data/mllib/sample_libsvm_data.txt")

# # Automatically identify categorical features, and index them.
# # Set maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(flight3)

# # Split the data into training and test sets (30% held out for testing)
# (trainingData, testData) = data.randomSplit([0.7, 0.3])

# Train a GBT model.
gbt = GBTRegressor(featuresCol="indexedFeatures", maxIter=10)

# Chain indexer and GBT in a Pipeline
pipeline = Pipeline(stages=[featureIndexer, gbt])

# Train model.  This also runs the indexer.
model = pipeline.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)

# Select example rows to display.
predictions.select("prediction", "label", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

gbtModel = model.stages[1]
print(gbtModel)  # summary only

In [ ]:
# display(flight.select("duration").show())
# display(flight2.select("duration_h", 'duration_m').show())
display(flight2.show(5))
# flight2.toPandas().head()

In [ ]:
import re
def getHours(x):
  return re.match('([0-9]+(?=h))', x)
temp = flight.select("duration").rdd.map(lambda x:getHours(x[0])).toDF()
temp.select("duration").show()

In [ ]:
flight.select("arr_time").show(2, truncate=False)

In [ ]:
# flight2 = (flight.withColumn('arr_time_local', flight.arr_time.substr(1, 23).cast('date'))                 
#           )

# flight3 = flight2.withColumn('arr_time', flight2.arr_time.cast('timestamp'))

# .select('arr_time').show(2, truncate=False)


# flight2 = (flight.withColumn('duration_minutes', flight.duration.substr(1, 23).cast('date'))                 
#           )

# flight.selectExpr("duration", "regexp_extract(duration,'([0-9]+(?=h))', 1) as duration_hour", "regexp_extract(duration,'([0-9]+(?=m))', 1)", ).show()
flight.selectExpr("regexp_extract(duration,'([0-9]+(?=h))', 1) as duration_hour").duration_hour.cast("double").show()

In [ ]:
flight2.select('arr_time', 'arr_time_local').dtypes
flight2.select('arr_time', 'arr_time_local').show(2, truncate=False)

In [ ]:
# from pyspark.sql import crosstab
# from pyspark.sql.functions import *

flight.crosstab('start_date','from_city_name').show()
# flight.columns


3
down vote
In order to simply cast a string column to a timestamp, the string column must be properly formatted.

To retrieve the "createdAt" column as a timestamp, you can write the UDF function that would convert the string

"2016-07-01T16:37:41-0400"
to

"2016-07-01 16:37:41"
and convert the "createdAt" column to a new format (don't forget to handle the timezone field).

Once you have a column containing timestamps as strings like "2016-07-01 16:37:41", a simple cast to timestamp would do the job, as you have it in your code.

You can read more about Date/Time/String Handling in Spark here.

In [ ]:
display(flight.select("arr_time").take(3))
display(flight2.select("arr_time").take(3))



In [ ]:
flight2.withColumn('local', F.from_utc_timestamp(flight2.arr_time, "AEST")).select("local").show()
flight2.withColumn('local', F.from_utc_timestamp(flight2.arr_time, "CTZ")).select("local").show()

In [ ]:
import time
from datetime import datetime

import pytz # $ pip install pytz
from tzlocal import get_localzone # $ pip install tzlocal



In [ ]:
# get local timezone    
local_tz = get_localzone() 
print(local_tz)
    

In [ ]:
# test it
# utc_now, now = datetime.utcnow(), datetime.now()
ts = time.time()
utc_now, now = datetime.utcfromtimestamp(ts), datetime.fromtimestamp(ts)
print(utc_now)
        

In [ ]:
local_now = utc_now.replace(tzinfo=pytz.utc).astimezone(local_tz) # utc -> local
print(local_now)


In [ ]:
assert local_now.replace(tzinfo=None) == now

In [ ]:
# test it
# utc_now, now = datetime.utcnow(), datetime.now()
ts = time.time()
utc_now, now = datetime.utcfromtimestamp(ts), datetime.fromtimestamp(ts)

local_now = utc_now.replace(tzinfo=pytz.utc).astimezone(local_tz) # utc -> local
assert local_now.replace(tzinfo=None) == now


* Spark-Sql doesn't support date-time, and nor timezones
* Using timestamp is the only solution
* from_unixtime(at) parses the epoch time correctly, just that the printing of it as a string changes it due to timezone. It is safe to assume that the  from_unixtime will convert it correctly ( although printing it might show different results)
* from_utc_timestamp will shift ( not just convert) the timestamp to that timezone, in this case it will subtract 8 hours to the time since (-08:00)
* printing sql results messes up the times with respect to timezone param
  	 	
* from_unixtime(at) does what from_utc_timestamp does too, it will parse a Unix timestamp integer (seconds since midnight 1970-01-01), and convert the time instant parsed from UTC to the system's default timezone. 


In [ ]:
flight2.selectExpr("from_utc_timestamp(arr_time, 'AEST') as testthis").show()

In [ ]:
flight2.dtypes
display(flight2.select('arr_time').take(3))
display(flight.select('arr_time').take(3))


In [ ]:
from pyspark.sql.functions import year, month, dayofmonth, hour, minute, weekofyear, crosstab

flight2.select(year("arr_time").alias('year'), 
               month("arr_time").alias('month'),
               dayofmonth("arr_time").alias('day'),
               hour("arr_time").alias('hour'),
               minute("arr_time").alias('minute'),
               weekofyear('arr_time').alias('week_no'),
               "arr_time_zone"
              ).show()



In [ ]:
from pyspark.sql.functions import regexp_extract, col

#flight.select(regexp_extract('arr_time', r'[+-][0-9]{2}:[0-9]{2}\b', 1)).alias('d').collect






In [ ]:
df = spark.createDataFrame([('2017-04-09T07:15:00.000+08:00',)], ['str'])
df.select(regexp_extract('str', '(\d+)-(\d+)', 1).alias('d')).collect()

In [ ]:
flight2.describe('price').show()

In [ ]:
from pyspark.sql.functions import *
# flight2.dtypes
# flight2.first()
display(flight2.select(max("start_date")).show())
display(flight2.select(min("start_date")).show())

In [ ]:
flight2 = flight2.withColumn('zero_price', flight2.)

In [ ]:
# flight.describe().show()
flight.describe("from_city_name").show() 

In [ ]:
from pyspark.sql.functions import colflight.filter(flight.price > 0).groupby(flight.search_date_x, flight.from_city_name, flight.to_city_name).count().show()

In [ ]:
from pyspark.sql.functions import col
# flight_to_brisbane = flight.where(col("price") > 0 & col("to_city_name") == "brisbane").groupby(flight.search_date_x).count()

In [ ]:
df.where(col("v").isin({"foo", "bar"})).count()

In [ ]:
numeric = sqlContext.createDataFrame([
    ('3.5,', '5.0', 'null'), ('2.0', '14.0', 'null'),  ('null', '38.0', 'null'),
    ('null', 'null', 'null'),  ('1.0', 'null', '4.0')],
    ('low', 'high', 'normal'))

numeric_filtered_1 = numeric.where(numeric['LOW'] != 'null')
numeric_filtered_1.show()

In [ ]:
flight.select('from_city_name', 'price').show(5)

h2o tutorial 
https://github.com/h2oai/h2o-tutorials/blob/master/tutorials/pysparkling/Chicago_Crime_Demo.md